# Mapas em Python [2/7]: Preparando o dataset

### O dataset possui informações sobre Boletins de Ocorrência na cidade de São Paulo. Nosso objetivo, neste exemplo, é organizar um subset para servir de base para construção de mapas.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import seaborn as sns

# Pedir para o pandas mostrar toda as as colunas do dataset
pd.options.display.max_columns = None

## 1) Importar o dataset

In [2]:
data = pd.read_csv('BO_2016.csv', low_memory=False)

In [3]:
data.head()

,NUM_BO,ANO_BO,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,DELEGACIA,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,NOME_DELEGACIA_CIRC,ANO,MES,FLAG_STATUS,RUBRICA,DESDOBRAMENTO,CONDUTA,LATITUDE,LONGITUDE,CIDADE,LOGRADOURO,NUMERO_LOGRADOURO,FLAG_STATUS.1,Unnamed: 21
0,3784,2016,10101,DECAP,DEL.SEC.1º CENTRO,01º D.P. SE,DECAP,DEL.SEC.1º CENTRO,78º D.P. JARDINS,2016,7,C,Furto (art. 155),NaN,TRANSEUNTE,-23.564984,-46.652035,S.PAULO,AVENIDA PAULISTA,1000,C,NaN
1,3426,2016,10102,DECAP,DEL.SEC.1º CENTRO,02º D.P. BOM RETIRO,DECAP,DEL.SEC.1º CENTRO,03º D.P. CAMPOS ELISEOS,2016,5,C,Roubo (art. 157),NaN,TRANSEUNTE,-23.542476,-46.641928,S.PAULO,PRAÇA DA REPUBLICA,0,C,NaN
2,6359,2016,10102,DECAP,DEL.SEC.1º CENTRO,02º D.P. BOM RETIRO,DECAP,DEL.SEC.1º CENTRO,03º D.P. CAMPOS ELISEOS,2016,10,C,Drogas sem autorização ou em desacordo (Art.33...,NaN,NaN,-23.542183,-46.640599,S.PAULO,RUA CONSELHEIRO NEBIAS,0,C,NaN
3,1267,2016,10103,DECAP,DEL.SEC.1º CENTRO,03º D.P. CAMPOS ELISEOS,DECAP,DEL.SEC.8º SAO MATEUS,49º D.P. SAO MATEUS,2016,3,C,Roubo (art. 157),NaN,CARGA,-23.609275,-46.455087,S.PAULO,RUA MADUREIRA CALHEIROS,15,C,NaN
4,4804,2016,10106,DECAP,DEL.SEC.1º CENTRO,06º D.P. CAMBUCI,DECAP,DEL.SEC.1º CENTRO,06º D.P. CAMBUCI,2016,12,C,Lesão corporal (art. 129),NaN,NaN,-23.573928,-46.620693,S.PAULO,RUA ROBERTSON,625,C,NaN


## 2) Breve análise exploratória

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774662 entries, 0 to 774661
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   NUM_BO                  774662 non-null  int64  
 1   ANO_BO                  774662 non-null  int64  
 2   ID_DELEGACIA            774662 non-null  int64  
 3   NOME_DEPARTAMENTO       774662 non-null  object 
 4   NOME_SECCIONAL          774662 non-null  object 
 5   DELEGACIA               774662 non-null  object 
 6   NOME_DEPARTAMENTO_CIRC  774662 non-null  object 
 7   NOME_SECCIONAL_CIRC     774662 non-null  object 
 8   NOME_DELEGACIA_CIRC     774662 non-null  object 
 9   ANO                     774662 non-null  int64  
 10  MES                     774662 non-null  int64  
 11  FLAG_STATUS             774662 non-null  object 
 12  RUBRICA                 774662 non-null  object 
 13  DESDOBRAMENTO           30092 non-null   object 
 14  CONDUTA             

In [ ]:
sns.heatmap(data.isnull(), cbar=False, yticklabels=False);

In [ ]:
data['RUBRICA'].value_counts().plot.barh(figsize=(10,8));

### Explorando e categorizando valores da coluna RUBRICA

In [ ]:
data['RUBRICA'].unique()

In [ ]:
# categoria furto
data['RUBRICA'] = data['RUBRICA'].replace(['Furto (art. 155)',
                                           'A.I.-Furto (art. 155)',
                                           'Furto de coisa comum (art. 156)',
                                           'A.I.-Furto de coisa comum (art. 156)',
                                           'Furto qualificado (art. 155, §4o.)',
                                           'A.I.-Furto qualificado (art. 155, §4o.)']
                                          ,'Furto')

# categoria roubo
data['RUBRICA'] = data['RUBRICA'].replace(['Roubo (art. 157)',
                                           'A.I.-Roubo (art. 157)',]
                                          ,'Roubo')

# categoria envolvendo drogas
data['RUBRICA'] = data['RUBRICA'].replace(['Drogas sem autorização ou em desacordo (Art.33, caput)',
                                           'A.I.-Drogas sem autorização ou em desacordo (Art.33, caput)',
                                           'Induzir, instigar ou auxiliar alguém ao uso indevido de droga(Art.33,§2º)',
                                           'A.I.-Tráfico de entorpecente (Art. 12)',
                                           'A.I.-Porte de entorpecente (Art. 16)',
                                           'A.I.-Oferecer droga a pessoa de seu relacionamento (Art.33,§3º)',
                                           'Oferecer droga a pessoa de seu relacionamento (Art.33,§3º)']
                                          ,'Crimes envolvendo drogas')

# categoria de lesões corporais
data['RUBRICA'] = data['RUBRICA'].replace(['Lesão corporal (art. 129)',
                                           'Lesão corporal (art 129 § 9º)',
                                           'Lesão corporal culposa (art. 129. §6o.)',
                                           'Lesão corporal de natureza GRAVE (art. 129, §1o.)',
                                           'A.I.-Lesão corporal de natureza GRAVE (art. 129, §1o.)',
                                           'A.I.-Lesão corporal (art 129 § 9º)',
                                           "Lesão corporal  de natureza 'GRAVÍSSIMA' (art. 129, §2o.)",
                                           "A.I.-Lesão corporal  de natureza 'GRAVÍSSIMA' (art. 129, §2o.)",
                                           'A.I.-Lesão corporal culposa (art. 129. §6o.)',
                                           'A.I.-Lesão corporal (art. 129)']
                                          ,'Lesão corporal')

# crimes sexuais
data['RUBRICA'] = data['RUBRICA'].replace(['Estupro (art.213)',
                                           'A.I.-Estupro de vulneravel (art.217-A)',
                                           'Estupro de vulneravel (art.217-A)']
                                          ,'Crime sexual')

# homicídios culposos
data['RUBRICA'] = data['RUBRICA'].replace(['A.I.-Homicídio culposo (art. 121, §3o.)',
                                           'Homicídio culposo (art. 121, §3o.)']
                                          ,'Crime sexual')

# homicídios dolosos
data['RUBRICA'] = data['RUBRICA'].replace(['A.I.-Homicídio qualificado (art. 121, §2o.)',
                                           'Homicídio qualificado (art. 121, §2o.)',
                                           'Homicídio simples (art. 121)',
                                           'A.I.-Homicídio simples (art. 121)',
                                           'Lesão corporal seguida de morte (art. 129, §3o.)']
                                          ,'Homicídio doloso')


# lesoes corporais envolvendo veículos
data['RUBRICA'] = data['RUBRICA'].replace(['Lesão corporal culposa na direção de veículo automotor (Art. 303)',
                                           'A.I.-Lesão corporal culposa na direção de veículo automotor (Art. 303)']
                                          ,'Lesão envolvendo veículos')

# homicídios envolvendo veículos
data['RUBRICA'] = data['RUBRICA'].replace(['A.I.-Homicídio culposo na direção de veículo automotor (Art. 302)',
                                           'Homicídio culposo na direção de veículo automotor (Art. 302)']
                                          ,'Homicídio envolvendo veículos')

In [ ]:
data['RUBRICA'].unique()

In [ ]:
data['RUBRICA'].value_counts()

In [ ]:
data['RUBRICA'].value_counts().plot.barh(figsize=(15,8));

In [ ]:
data['DELEGACIA'].value_counts().nlargest(50)

In [ ]:
data['DELEGACIA'].value_counts().nlargest(10).plot.barh(figsize=(12,12*0.45));

In [ ]:
data['MES'].value_counts(ascending=True).sort_index()

In [ ]:
plt.figure(figsize=(8*1.68,8))
sns.countplot(data=data, x='MES', color='Blue');

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,15*0.45))
fig.suptitle('Contagem de ocorrências por mês')

ax1.plot(data['MES'].value_counts().sort_index())
ax1.set_ylim(0,80000)
ax1.set_xlim(1,12)
ax1.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
ax1.set(xlabel='mês', ylabel='quantidade', title='Ocorrências por mês');

ax2.plot(data['MES'].value_counts().sort_index())
ax2.set_ylim(60000,80000)
ax2.set_xlim(1,12)
ax2.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
ax2.set(xlabel='mês', ylabel='quantidade', title='Detalhe das ocorrências por mês');

In [ ]:
# cálculo de quanto evoluiu em relação ao mês passado (em percentagem)
# mudança percentual a partir do valor anterior, portanto o mês 2 em relação ao mês 3
# 'aceleração percentual' na diferença da quantidade de ocorrências reportadas em B.O.s

contagem_dados_mes = data['MES'].value_counts(ascending=True).sort_index()

contagem_dados_mes.pct_change()

In [ ]:
# plotando em gráfico, indicando linha base 0 para melhor visualização

plt.figure(figsize=(8*1.68,8))

ax = contagem_dados_mes.pct_change().plot()
ax.axhline(0, ls='--', color='red')
ax.set_xlim(1,12)

plt.xticks(np.arange(1,13,1)); #setando valores do eixo X, entre 1 e 13, de um em um

# sns.despine(offset=10, trim=True);

## 3) Manipular o dataset
### Selecionando e ordenando colunas que irão compor meu dataset para georreferenciar - operação alinhada com o objetivo de identificar no mapa as ocorrências

In [ ]:
data_geo = data[['NUM_BO','DELEGACIA','RUBRICA','MES','LATITUDE','LONGITUDE']]

In [ ]:
data_geo

### Limpando linhas cujo valor na coluna 'LATITUDE' é nulo

In [ ]:
sns.heatmap(data_geo.isnull(), cbar=False, yticklabels=False);

In [ ]:
data_geo = data_geo.dropna(subset=['LATITUDE'])

In [ ]:
sns.heatmap(data_geo.isnull(), cbar=False, yticklabels=False);

### Selecionando uma amostra estatisticamente proporcional

In [ ]:
# usar o sample pode ser uma alternativa, porém queremos uma amostra proporcional, o que o sample não oferece

# data_geo = data_geo.sample(1500)

In [ ]:
pd.DataFrame(columns=data_geo.columns)

In [ ]:
amostra = pd.DataFrame(columns=data_geo.columns)

In [ ]:
# algorítimo para pegar amostra proporcional dos meses (proporcionalidade 2 para 1000)

for mes in range(1,13,1):
    amostra = amostra.append(data_geo[data_geo['MES'] == mes].sample(int(len(data_geo[data_geo['MES'] == mes])*0.002)))

In [ ]:
amostra

## Organizando a visualização do que fizemos acima

In [ ]:
total = data_geo['MES'].value_counts().sort_index()

amostragem = amostra['MES'].value_counts().sort_index()

percent_amostra = 100*(amostra['MES'].value_counts()/data_geo['MES'].value_counts())

In [ ]:
comparacao = {'Total': total, 'Amostra': amostragem, '% amostra': percent_amostra}

pd.DataFrame.from_dict(comparacao)

## Salvando a amostra em documento csv

In [ ]:
amostra.to_csv('amostra_data_geo_bo2016.csv')